# Mastermap
------
Mastermap is an Enrichment map that contains more than 2 enrichment analyses.

![Pandey mastermap example](./gsea_mastermap.png)

## Input

1. ***Expression file (Optional)*** - a file with absolute or relative expression values for a set of genes and experiments.
  * Two different options:
    * One expression file where each column represents an individual experiment and there are N columns (one for each of the N experiments.)
    * N expression files where each file contains the expression for each experiment (possibly with ctrl vs treatment values)
    
1. ***Enrichment Results***
  * Two different options:
    * GSEA results
      * a GMT file
      * N GSEA output directories with enrichment results
    * Gprofiler/David/Bingo results
      * N enrichment results files.
      
1. ***User specified parameters***:
  * p-value threshold (Optional)
  * q-value threshold
  * Minimum number of experiments pathway needs to appear in for it to be included in output
  * NES threshold (Optional, GSEA speciric, not sure if it is required)

------

## Generating a Mastermap in the absence of an app.

### Collate all the GSEA enrichment results.

1. Place all the GSEA enrichment folders into one directory.
1. Navigate to that directory and run the following shell script
<pre>
`
    #go through each directory
    for dir in `ls -d *.GseaPreranked.*`; do
        cd $dir
        NAME1=`echo $dir | cut -d'.' -f1 | cut -d '_' -f1`
        NAME2=`echo $dir | cut -d'.' -f1 | cut -d '_' -f2`
        NAME=${NAME1}_${NAME2}
        #append the name to the sea results file
        for file in `ls gsea_report*.xls`; do		
            awk -v name="${NAME}" '{print name"\t", $0}' $file > ${file}_forR
            #remove the first line
            tail +2 ${file}_forR > temp.txt
            mv temp.txt ${file}_forR
        done
        mv *_forR ../
        cd ..
    done
    #merge all results into one file
    cat *_forR > Gsea_reports_alltissues.txt
    rm *_forR
`
</pre>
1. The above shell script will go into each of the GSEA directories (based on the presence of ".GseaPreranked." in the folder name.  If GSEA was not run with preranked option then swap ".GseaPreranked." for ".Gsea.") and append the folder name (minus the GSEA added analysis type and random number) as the first column to the gsea positive and negative results file.  The files for each of the folders is appended to create the file   
  * GSEA results files contain the following headers:
    1. NAME
    1. GS - link to MSigDB
    1. GS DETAILs
    1. SIZE
    1. ES
    1. NES
    1. NOM p-val
    1. FDR q-val
    1. FWER p-val
    1. RANK AT MAX
    1. LEADING EDGE
  * Resulting "Gsea_reports_alltisues.txt" file contains the same headers with the addition of one column in front of Name
    1. Analysis name - taken from the name of the GSEA results directory.

### User defined thresholds

In [1]:
pval_thresh <- 0.01
fdr_thresh <- 0.01
NES_thresh <- 0
min_experiments <- 3

similarity_cutoff <- "0.25"
similarity_metric <- "JACCARD"

### Load the file containing the collated GSEA results

In [2]:
#set working directory
setwd("./One_expression_file_example/GSEA_results/");

In [9]:
gsea_enrichments <- read.table("Gsea_reports_alltissues.txt", header = TRUE, sep = "\t",as.is = TRUE, quote="\"")

colnames(gsea_enrichments) <- c("Experiment","Name","GS","GS DETAILS","SIZE","ES", "NES",
                                "NOM p-val","FDR q-val","FWER p-val","RANK AT MAX","LEADING EDGE");


#filter gsea enrichment by thresholds
#only include NES scores that are > 0 (for the proteomics data the under-enriched is not fitting for this analysis)
gsea_enrichments_filtered <- gsea_enrichments[which(gsea_enrichments[,'NOM p-val']<=pval_thresh & 
                                                    gsea_enrichments[,'FDR q-val']<=fdr_thresh & 
                                                    gsea_enrichments[,'NES']>NES_thresh),]

#row_names - get the unique set of pathways that is contained in the collated data.  column 2 indicates the pathway name
row_names <- unique(gsea_enrichments_filtered[,2])
#column_names - get the unique set of experiments contains in the collated data.  column 1 indicates the experiment type
column_names <- unique(gsea_enrichments_filtered[,1])

#create a matrix which will store the tissue profiles for all genesets in the thresholded set
pathways2experiments_significant <- matrix(nrow=length(row_names), ncol=length(column_names),dimnames=list(row_names, column_names))
pathways2experiments_all <- matrix(nrow=length(row_names), ncol=length(column_names),dimnames=list(row_names, column_names))

for (i in 1:length(row_names)){
  for (j in 1:length(column_names)){
      #only add the NES value if this pathway is significant for this experiment
    if(length(which(gsea_enrichments_filtered[,1] == column_names[j] & 
                    gsea_enrichments_filtered[,2] == row_names[i])) > 0 ){
        pathways2experiments_significant[i,j] = gsea_enrichments_filtered[which(gsea_enrichments_filtered[,1] == column_names[j] & 
                                                               gsea_enrichments_filtered[,2] == row_names[i]), 'NES']   
    }
      #only add the NES value if it exists in the enrichments (irrespective of significance)
      if(length(which(gsea_enrichments[,1] == column_names[j] & 
                    gsea_enrichments[,2] == row_names[i])) > 0 ){
      pathways2experiments_all[i,j] = gsea_enrichments[which(gsea_enrichments[,1] == column_names[j] & 
                                                               gsea_enrichments[,2] == row_names[i]), 'NES']
      }
  }
}

# only include the pathways that are significant in at least X experiments
pathways2experiments_significant <- pathways2experiments_significant[
    which(apply(pathways2experiments_significant,1, function(x){length(which(x!=0))}) >= min_experiments),]
pathways2experiments_all <- pathways2experiments_all[
    which(apply(pathways2experiments_significant,1, function(x){length(which(x!=0))}) >= min_experiments),]





# create a fake gsea summary enrichment file.  Can't use generic without loading in the gmt file and computing genes
# that belong to each set.  In order to use gsea option enrichment file needs to be in the same format as gsea.

#create a collapsed enrichment file
# NES and ES are just the summed values
# p-value and q-value is the average
collapsedenr_column_names = c("Name","GS","GS DETAILS","SIZE","ES", "NES","NOM p-val","FDR q-val","FWER p-val","RANK AT MAX","LEADING EDGE");

#limit to a subset
row_names_subset <- rownames(pathways2experiments_significant)[
    which(apply(pathways2experiments_significant,1, function(x){length(which(x!=0))}) >= min_experiments)]

collapsed_enrichments<- matrix(nrow=length(row_names_subset), ncol=length(collapsedenr_column_names),
                               dimnames=list(row_names_subset, collapsedenr_column_names))
#go through the genesets
for (i in 1:length(row_names_subset)){
  #get all the genesets from the filtered set
  indices <- which(gsea_enrichments_filtered[,2] == row_names_subset[i])
  subset <- gsea_enrichments_filtered[indices,];
  #grab the first name,gs,gs details, size - they are all the same 
  collapsed_enrichments[i,1] <- subset[1,2]
  collapsed_enrichments[i,2] <- subset[1,3]
  collapsed_enrichments[i,3] <- subset[1,4]
  collapsed_enrichments[i,4] <- subset[1,5]
  #get the summed ES score
  collapsed_enrichments[i,5] <- sum(subset[,6])
  #get the summed NES score
  collapsed_enrichments[i,6] <- sum(subset[,7])
  #get the average pvalue
  collapsed_enrichments[i,7] <- 0.01
  #get the average fdr
  collapsed_enrichments[i,8] <- 0.01
  #get the average FWER
  collapsed_enrichments[i,9] <- mean(subset[,10])
  #get the first rank at max
  collapsed_enrichments[i,10] <- max(subset[,11])
  collapsed_enrichments[i,11] <- subset[1,12]

}


### Get rid of the blank spaces in the pathway names

In [14]:
collapsed_enrichments[,1] <- trimws(collapsed_enrichments[,1])
collapsed_enrichments[,2] <- trimws(collapsed_enrichments[,2])
rownames(collapsed_enrichments) <- trimws(rownames(collapsed_enrichments))

### Output files that could have been used by vista clara (in previous version of cytoscape) and in current version charts

In [38]:
#output the pathways2experiments_all
#
# The pathways2experiments file is a matrix of pathways to experiments where each value in the matrix is a significant NES value.
# This table can be used to calculate which genesets pass the minimum expereiment threshold but should not be used 
# as an expression file for vista clara plugin as it is missing NES values for pathways and experiments that were not significant

pathways2experiments_all[is.na(pathways2experiments_all)] <- 0
#we are not interested in the sets that are significant in rest of the tissues.
pathways2experiments_all[which(pathways2experiments_all < 0)] <- 0
rownames(pathways2experiments_all) <- trimws(rownames(pathways2experiments_all))
output_pathways2experiments_all <- cbind(rownames(pathways2experiments_all), pathways2experiments_all)

output_pathways2experiments_all <- output_pathways2experiments_all[grep(output_pathways2experiments_all[,1],
                                              pattern = "TCR SIGNALING IN NA",
                                              invert=TRUE),]
output_pathways2experiments_all<- output_pathways2experiments_all[grep(output_pathways2experiments_all[,1],
                                              pattern = "LOSS OF PROTEINS REQUIRED FOR INTERPHASE MICROTUBULE",
                                              invert=TRUE),]
output_pathways2experiments_all<- output_pathways2experiments_all[grep(output_pathways2experiments_all[,1],
                                              pattern = "DOWNSTREAM SIGNALING IN NA",
                                              invert=TRUE),]
write.table(output_pathways2experiments_all, file="pathways2experiments_all.txt", sep="\t", 
            row.names=FALSE, col.names=TRUE,quote=FALSE)


#output the pathways2experiments_significant as well
#

pathways2experiments_significant[is.na(pathways2experiments_significant)] <- 0
rownames(pathways2experiments_significant) <- trimws(rownames(pathways2experiments_significant))
output_pathways2experiments_significant <- cbind(rownames(pathways2experiments_significant), pathways2experiments_significant)

output_pathways2experiments_significant <- output_pathways2experiments_significant[grep(output_pathways2experiments_significant[,1],
                                              pattern = "TCR SIGNALING IN NA",
                                              invert=TRUE),]
output_pathways2experiments_significant<- output_pathways2experiments_significant[grep(output_pathways2experiments_significant[,1],
                                              pattern = "LOSS OF PROTEINS REQUIRED FOR INTERPHASE MICROTUBULE",
                                              invert=TRUE),]
output_pathways2experiments_significant <- output_pathways2experiments_significant[grep(output_pathways2experiments_significant[,1],
                                              pattern = "DOWNSTREAM SIGNALING IN NA",
                                              invert=TRUE),]

write.table(output_pathways2experiments_significant, file="pathways2experiments_significant.txt", sep="\t", 
            row.names=FALSE, col.names=TRUE,quote=FALSE)


## Get rid of genesets that get messed up in R because of weird characters in their name

In [3]:
#to save on computation you can reload previous computed collapsed enrichments
#enrichment_results_file_name <- paste("mastermap_enrichments_fdr", fdr_thresh, "_minexp", min_experiments,".txt", sep="")
#collapsed_enrichments <- read.table(enrichment_results_file_name, , header = TRUE, sep = "\t",as.is = TRUE, quote="\"")

In [36]:
Sys.getlocale()
Sys.setlocale(locale="C")

[1] "LC_CTYPE=C;LC_NUMERIC=C;LC_TIME=C;LC_COLLATE=C;LC_MONETARY=C;LC_MESSAGES=en_US.UTF-8;LC_PAPER=en_US.UTF-8;LC_NAME=C;LC_ADDRESS=C;LC_TELEPHONE=C;LC_MEASUREMENT=en_US.UTF-8;LC_IDENTIFICATION=C"

[1] "LC_CTYPE=C;LC_NUMERIC=C;LC_TIME=C;LC_COLLATE=C;LC_MONETARY=C;LC_MESSAGES=en_US.UTF-8;LC_PAPER=en_US.UTF-8;LC_NAME=C;LC_ADDRESS=C;LC_TELEPHONE=C;LC_MEASUREMENT=en_US.UTF-8;LC_IDENTIFICATION=C"

In [17]:
collapsed_enrichments <- collapsed_enrichments[grep(collapsed_enrichments[,1],
                                              pattern = "TCR SIGNALING IN NA",
                                              invert=TRUE),]
collapsed_enrichments <- collapsed_enrichments[grep(collapsed_enrichments[,1],
                                              pattern = "LOSS OF PROTEINS REQUIRED FOR INTERPHASE MICROTUBULE",
                                              invert=TRUE),]
collapsed_enrichments <- collapsed_enrichments[grep(collapsed_enrichments[,1],
                                              pattern = "DOWNSTREAM SIGNALING IN NA",
                                              invert=TRUE),]

In [18]:
enrichment_results_file_name <- paste("mastermap_enrichments_fdr", fdr_thresh, "_minexp", min_experiments,".txt", sep="")
write.table( collapsed_enrichments, file=enrichment_results_file_name, sep="\t", row.names=FALSE, col.names=TRUE,quote=FALSE)

## Initialize cytoscape

In [31]:
library(RJSONIO)
library(httr)

port.number=1234
base.url = paste("http://localhost:",toString(port.number),"/v1", sep="")

print(base.url)

version.url = paste(base.url,"version", sep="/")
cytoscape.version = GET(version.url)
cy.version = fromJSON(rawToChar(cytoscape.version$content))
print(cy.version)

[1] "http://localhost:1234/v1"
      apiVersion cytoscapeVersion 
            "v1"          "3.4.0" 


In [32]:
enrichmentmap.url <- paste(base.url, "commands", "enrichmentmap", "build", sep="/")
#mac file paths
gmt_file="/Users/risserlin/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/GSEA_results/Human_AllPathwaysGOBP_noiea_May_14_2013_symbol.gmt"
path_to_file="/Users/risserlin/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/GSEA_results/"
exp_file="/Users/risserlin/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/Human_Proteome_Map_spectral_count_gene_tissue.txt"
#windows file paths
#gmt_file="C:/Users/zaphod/Ruth_dropbox/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/GSEA_results/Human_AllPathwaysGOBP_noiea_May_14_2013_symbol.gmt"
#exp_file="C:/Users/zaphod/Ruth_dropbox/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/Human_Proteome_Map_spectral_count_gene_tissue.gct"
#path_to_file="C:/Users/zaphod/Ruth_dropbox/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/GSEA_results/"

## Create Mastermap

In [33]:
enr_file = paste(path_to_file,enrichment_results_file_name,sep="")

em_params <- list(analysisType = "gsea",enrichmentsDataset1 = enr_file,pvalue="1.0",qvalue="0.05",
                  expressionDataset1 = exp_file, gmtFile = gmt_file,
                  similaritycutoff=similarity_cutoff,coeffecients=similarity_metric)

response <- GET(url=enrichmentmap.url, query=em_params)

In [34]:
content(response, "text",encoding="ISO-8859-1")

[1] "Finished\n"

## Create a different version of Mastermap with just the Fetal samples

In [49]:
min_experiments <- 3
fetal_subset <- gsea_enrichments[grep(pattern = "Fetal", gsea_enrichments[,1]),]

#filter gsea enrichment by thresholds
#only include NES scores that are > 0 (for the proteomics data the under-enriched is not fitting for this analysis)
fetal_subset_filtered <- fetal_subset[which(fetal_subset[,'NOM p-val']<=pval_thresh & 
                                                    fetal_subset[,'FDR q-val']<=fdr_thresh & 
                                                    fetal_subset[,'NES']>NES_thresh),]

#row_names - get the unique set of pathways that is contained in the collated data.  column 2 indicates the pathway name
row_names <- unique(fetal_subset_filtered[,2])
#column_names - get the unique set of experiments contains in the collated data.  column 1 indicates the experiment type
column_names <- unique(fetal_subset_filtered[,1])

#create a matrix which will store the tissue profiles for all genesets in the thresholded set
fetal_subset_pathways2experiments_significant <- matrix(nrow=length(row_names), ncol=length(column_names),dimnames=list(row_names, column_names))
fetal_subset_pathways2experiments_all <- matrix(nrow=length(row_names), ncol=length(column_names),dimnames=list(row_names, column_names))

for (i in 1:length(row_names)){
  for (j in 1:length(column_names)){
      #only add the NES value if this pathway is significant for this experiment
    if(length(which(fetal_subset_filtered[,1] == column_names[j] & 
                    fetal_subset_filtered[,2] == row_names[i])) > 0 ){
        fetal_subset_pathways2experiments_significant[i,j] = fetal_subset_filtered[which(fetal_subset_filtered[,1] == column_names[j] & 
                                                               fetal_subset_filtered[,2] == row_names[i]), 'NES']   
    }
      #only add the NES value if it exists in the enrichments (irrespective of significance)
      if(length(which(fetal_subset[,1] == column_names[j] & 
                    fetal_subset[,2] == row_names[i])) > 0 ){
      fetal_subset_pathways2experiments_all[i,j] = fetal_subset[which(fetal_subset[,1] == column_names[j] & 
                                                               fetal_subset[,2] == row_names[i]), 'NES']
      }
  }
}

# only include the pathways that are significant in at least X experiments
fetal_subset_pathways2experiments_significant <- fetal_subset_pathways2experiments_significant[
    which(apply(fetal_subset_pathways2experiments_significant,1, function(x){length(which(x!=0))}) >= min_experiments),]
fetal_subset_pathways2experiments_all <- fetal_subset_pathways2experiments_all[
    which(apply(fetal_subset_pathways2experiments_significant,1, function(x){length(which(x!=0))}) >= min_experiments),]

# create a fake gsea summary enrichment file.  Can't use generic without loading in the gmt file and computing genes
# that belong to each set.  In order to use gsea option enrichment file needs to be in the same format as gsea.

#create a collapsed enrichment file
# NES and ES are just the summed values
# p-value and q-value is the average
collapsedenr_column_names = c("Name","GS","GS DETAILS","SIZE","ES", "NES","NOM p-val","FDR q-val","FWER p-val","RANK AT MAX","LEADING EDGE");

#limit to a subset
row_names_subset <- rownames(fetal_subset_pathways2experiments_significant)[
    which(apply(fetal_subset_pathways2experiments_significant,1, function(x){length(which(x!=0))}) >= min_experiments)]

collapsed_enrichments<- matrix(nrow=length(row_names_subset), ncol=length(collapsedenr_column_names),
                               dimnames=list(row_names_subset, collapsedenr_column_names))
#go through the genesets
for (i in 1:length(row_names_subset)){
  #get all the genesets from the filtered set
  indices <- which(fetal_subset_filtered[,2] == row_names_subset[i])
  subset <- fetal_subset_filtered[indices,];
  #grab the first name,gs,gs details, size - they are all the same 
  collapsed_enrichments[i,1] <- subset[1,2]
  collapsed_enrichments[i,2] <- subset[1,3]
  collapsed_enrichments[i,3] <- subset[1,4]
  collapsed_enrichments[i,4] <- subset[1,5]
  #get the summed ES score
  collapsed_enrichments[i,5] <- sum(subset[,6])
  #get the summed NES score
  collapsed_enrichments[i,6] <- sum(subset[,7])
  #get the average pvalue
  collapsed_enrichments[i,7] <- 0.01
  #get the average fdr
  collapsed_enrichments[i,8] <- 0.01
  #get the average FWER
  collapsed_enrichments[i,9] <- mean(subset[,10])
  #get the first rank at max
  collapsed_enrichments[i,10] <- max(subset[,11])
  collapsed_enrichments[i,11] <- subset[1,12]

}

collapsed_enrichments[,1] <- trimws(collapsed_enrichments[,1])
collapsed_enrichments[,2] <- trimws(collapsed_enrichments[,2])
rownames(collapsed_enrichments) <- trimws(rownames(collapsed_enrichments))

collapsed_enrichments <- collapsed_enrichments[grep(collapsed_enrichments[,1],
                                              pattern = "TCR SIGNALING IN NA",
                                              invert=TRUE),]
collapsed_enrichments <- collapsed_enrichments[grep(collapsed_enrichments[,1],
                                              pattern = "LOSS OF PROTEINS REQUIRED FOR INTERPHASE MICROTUBULE",
                                              invert=TRUE),]
collapsed_enrichments <- collapsed_enrichments[grep(collapsed_enrichments[,1],
                                              pattern = "DOWNSTREAM SIGNALING IN NA",
                                              invert=TRUE),]

enrichment_results_file_name <- paste("Fetal_only_mastermap_enrichments_fdr", fdr_thresh, "_minexp", min_experiments,".txt", sep="")
write.table( collapsed_enrichments, file=enrichment_results_file_name, sep="\t", row.names=FALSE, col.names=TRUE,quote=FALSE)

In [50]:
#output the pathways2experiments_all
#
# The pathways2experiments file is a matrix of pathways to experiments where each value in the matrix is a significant NES value.
# This table can be used to calculate which genesets pass the minimum expereiment threshold but should not be used 
# as an expression file for vista clara plugin as it is missing NES values for pathways and experiments that were not significant

fetal_subset_pathways2experiments_all[is.na(fetal_subset_pathways2experiments_all)] <- 0
#we are not interested in the sets that are significant in rest of the tissues.
fetal_subset_pathways2experiments_all[which(fetal_subset_pathways2experiments_all < 0)] <- 0
rownames(fetal_subset_pathways2experiments_all) <- trimws(rownames(fetal_subset_pathways2experiments_all))
output_pathways2experiments_all <- cbind(rownames(fetal_subset_pathways2experiments_all), 
                                         fetal_subset_pathways2experiments_all)

output_pathways2experiments_all <- output_pathways2experiments_all[grep(output_pathways2experiments_all[,1],
                                              pattern = "TCR SIGNALING IN NA",
                                              invert=TRUE),]
output_pathways2experiments_all<- output_pathways2experiments_all[grep(output_pathways2experiments_all[,1],
                                              pattern = "LOSS OF PROTEINS REQUIRED FOR INTERPHASE MICROTUBULE",
                                              invert=TRUE),]
output_pathways2experiments_all<- output_pathways2experiments_all[grep(output_pathways2experiments_all[,1],
                                              pattern = "DOWNSTREAM SIGNALING IN NA",
                                              invert=TRUE),]
write.table(output_pathways2experiments_all, file=paste("fetal_subset_pathways2experiments_all","_minexp", min_experiments,".txt",sep=""), sep="\t", 
            row.names=FALSE, col.names=TRUE,quote=FALSE)


#output the pathways2experiments_significant as well
#

fetal_subset_pathways2experiments_significant[is.na(fetal_subset_pathways2experiments_significant)] <- 0
rownames(fetal_subset_pathways2experiments_significant) <- trimws(rownames(fetal_subset_pathways2experiments_significant))
output_pathways2experiments_significant <- cbind(rownames(fetal_subset_pathways2experiments_significant), 
                                                 fetal_subset_pathways2experiments_significant)

output_pathways2experiments_significant <- output_pathways2experiments_significant[grep(output_pathways2experiments_significant[,1],
                                              pattern = "TCR SIGNALING IN NA",
                                              invert=TRUE),]
output_pathways2experiments_significant<- output_pathways2experiments_significant[grep(output_pathways2experiments_significant[,1],
                                              pattern = "LOSS OF PROTEINS REQUIRED FOR INTERPHASE MICROTUBULE",
                                              invert=TRUE),]
output_pathways2experiments_significant <- output_pathways2experiments_significant[grep(output_pathways2experiments_significant[,1],
                                              pattern = "DOWNSTREAM SIGNALING IN NA",
                                              invert=TRUE),]

write.table(output_pathways2experiments_significant, file=paste("fetal_subset_pathways2experiments_significant","_minexp", min_experiments,".txt",sep=""), sep="\t", 
            row.names=FALSE, col.names=TRUE,quote=FALSE)


In [47]:
enr_file = paste(path_to_file,enrichment_results_file_name,sep="")

em_params <- list(analysisType = "gsea",enrichmentsDataset1 = enr_file,pvalue="1.0",qvalue="0.05",
                  expressionDataset1 = exp_file, gmtFile = gmt_file,
                  similaritycutoff=similarity_cutoff,coeffecients=similarity_metric)

response <- GET(url=enrichmentmap.url, query=em_params)